# ETL BBDD EDlist
El objetivo de esta ETL es **extraer** las 3 listas que tiene la Endocrine Disruptor List https://edlists.org/ **transformar** cada BBDD para luego unir las 3 listas y posteriormente **cargar** usar este dataset transformado para otros fines que explicaremos más adelante. 

Para ello tenenmos que tener en cuenta que en la EDlist se explican los 3 listados: https://edlists.org/the-ed-lists

The three lists are:
* List I: Substances identified as endocrine disruptors at EU level
* List II: Substances under evaluation for endocrine disruption under an EU legislation
* List III: Substances considered, by the evaluating National Authority, to have endocrine disrupting properties

Tambien explica que con respecto a la lista III: "It should be noted that these compounds are not necessarily considered as suspected EDs at the EU level."

Con esta información, determinamos para nuestro análisis que: 
1. Lo que está en la lista I, solo debería estar en esta porque ya ha pasado de la lista II de evaluación o de la lista III en la que algún país miembro lo considera ED, pero el resto de la UE no. 
2. La lista II, puede contener ingredientes de la lista I que pese a estar en la lista I no se hayan eliminado de esta, o ingredientes de la lista III porque sigue en evaluación por parde de la UE, independientemente de que el país miembro que lo ha propuesto lo considere ED en su país. Si aparece en el archivo de esta lista las otras dos listas daremos la siguiente prioridad: lista I > lista II > lista III.
3. Lo que esté en la lista III, puede estar n la lista I pero aun no se haya eliminado de esta lista o tambien en la lista II porque esté en evaluación por la UE. Por tanto seguiremos la misma prioridad que en el apartado anterior.

## 1. Importamos librerias 

In [1]:
import pandas as pd
import numpy as np

## 2. Lectura de datos y estructuras de dataframes

In [2]:
# leemos los datasets de EDlist
edlist_1 = pd.read_excel("../../data/raw/list1.xlsx")
edlist_2 = pd.read_excel("../../data/raw/list2.xlsx")
edlist_3 = pd.read_excel("../../data/raw/list3.xlsx")

In [3]:
print (u"""
Nº de filas del dataset Edlist_1: {}
Nº de columnas del dataset Edlist_1: {}
Las variables de Edlist_1 son: {}

Nº de filas del dataset Edlist_2: {}
Nº de columnas del dataset Edlist_2: {}
Las variables de Edlist_2 son: {}

Nº de filas del dataset Edlist_3: {}
Nº de columnas del dataset Edlist_3: {}
Las variables de Edlist_3 son: {}

""".format(
    edlist_1.shape[0], edlist_1.shape[1], edlist_1.columns,
    edlist_2.shape[0], edlist_2.shape[1], edlist_2.columns,
    edlist_3.shape[0], edlist_3.shape[1], edlist_3.columns
    ))


Nº de filas del dataset Edlist_1: 226
Nº de columnas del dataset Edlist_1: 10
Las variables de Edlist_1 son: Index(['ID (for name)', 'Name and abbreviation', 'CAS no.', 'EC / List no.',
       'Health effects', 'Environmental effects', 'Status', 'Regulatory Field',
       'Status year', 'Appears on lists'],
      dtype='object')

Nº de filas del dataset Edlist_2: 104
Nº de columnas del dataset Edlist_2: 10
Las variables de Edlist_2 son: Index(['ID (for name)', 'Name and abbreviation', 'CAS no.', 'EC / List no.',
       'Health effects', 'Environmental effects', 'Status', 'Year',
       'Regulatory Field', 'Appears on lists'],
      dtype='object')

Nº de filas del dataset Edlist_3: 33
Nº de columnas del dataset Edlist_3: 10
Las variables de Edlist_3 son: Index(['ID (for name)', 'Name and abbreviation', 'CAS no.', 'EC / List no.',
       'Health effects', 'Environmental effects', 'Year', 'Status',
       'Regulatory Field', 'Also appears on lists'],
      dtype='object')




In [4]:
edlist_1.head(2)

,ID (for name),Name and abbreviation,CAS no.,EC / List no.,Health effects,Environmental effects,Status,Regulatory Field,Status year,Appears on lists
0,358,"(±)-1,7,7-trimethyl-3-[(4-methylphenyl)methyle...",36861-47-9,253-242-6,Yes,NaN,Commission EDC list,Cosmetics,2019,List I
1,358,"(±)-1,7,7-trimethyl-3-[(4-methylphenyl)methyle...",36861-47-9,253-242-6,Yes,NaN,List III National Authority evaluation,NaN,2021,List I


In [5]:
edlist_2.head(2)

,ID (for name),Name and abbreviation,CAS no.,EC / List no.,Health effects,Environmental effects,Status,Year,Regulatory Field,Appears on lists
0,122,"6,6'-di-tert-butyl-4,4'-thiodi-m-cresol",96-69-5,202-525-2,Yes,NaN,Corap List,2015,REACH,List II
1,141,Dicyclohexyl phthalate (DCHP),84-61-7,201-545-9,NaN,Yes,Corap List,2017,REACH,"List I, List II"


In [6]:
edlist_3.head(2)

,ID (for name),Name and abbreviation,CAS no.,EC / List no.,Health effects,Environmental effects,Year,Status,Regulatory Field,Also appears on lists
0,123,Tris(methylphenyl)phosphate,1330-78-5,215-548-8,Yes,NaN,2020,List III National Authority evaluation,NaN,List III
1,143,Bisphenol AF,1478-61-1,216-036-7,Yes,NaN,2020,List III National Authority evaluation,NaN,List III


In [7]:
edlist_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226 entries, 0 to 225
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ID (for name)          226 non-null    int64 
 1   Name and abbreviation  226 non-null    object
 2   CAS no.                211 non-null    object
 3   EC / List no.          166 non-null    object
 4   Health effects         64 non-null     object
 5   Environmental effects  182 non-null    object
 6   Status                 226 non-null    object
 7   Regulatory Field       222 non-null    object
 8   Status year            226 non-null    int64 
 9   Appears on lists       226 non-null    object
dtypes: int64(2), object(8)
memory usage: 17.8+ KB


In [8]:
edlist_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ID (for name)          104 non-null    int64 
 1   Name and abbreviation  104 non-null    object
 2   CAS no.                100 non-null    object
 3   EC / List no.          102 non-null    object
 4   Health effects         77 non-null     object
 5   Environmental effects  67 non-null     object
 6   Status                 104 non-null    object
 7   Year                   104 non-null    object
 8   Regulatory Field       99 non-null     object
 9   Appears on lists       104 non-null    object
dtypes: int64(1), object(9)
memory usage: 8.3+ KB


In [9]:
edlist_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ID (for name)          33 non-null     int64 
 1   Name and abbreviation  33 non-null     object
 2   CAS no.                33 non-null     object
 3   EC / List no.          33 non-null     object
 4   Health effects         31 non-null     object
 5   Environmental effects  12 non-null     object
 6   Year                   33 non-null     object
 7   Status                 33 non-null     object
 8   Regulatory Field       20 non-null     object
 9   Also appears on lists  33 non-null     object
dtypes: int64(1), object(9)
memory usage: 2.7+ KB


## 3. Consulta de conflicto entre listas y mapeo de variables
Como se ha adelantado en el enunciado hay componentes que aparecen en otras listas, para ello consultamos la variable "Appears on lists"

In [10]:
# Consultamos los valores de "Appears on lists"
print("Edlist_1:")
print(edlist_1['Appears on lists'].value_counts(dropna= False))
print("\nEdlist_2:")
print(edlist_2['Appears on lists'].value_counts(dropna= False))
print("\nEdlist_3:")
print(edlist_3['Also appears on lists'].value_counts(dropna= False))

Edlist_1:
Appears on lists
List I             223
List I, List II      3
Name: count, dtype: int64

Edlist_2:
Appears on lists
List II              79
List II, List III    22
List I, List II       3
Name: count, dtype: int64

Edlist_3:
Also appears on lists
List II, List III    22
List III             11
Name: count, dtype: int64


- En la lista I: hay 3 casos
- En la lista II: hay 22+3= 25 casos
- En la lista III: hay 22 casos

In [11]:
# Vemos los casos concreto de edlist_1
conflictos_1 = edlist_1[edlist_1['Appears on lists'].str.contains(",", na=False)]
print("Conflictos en Edlist_1:")
conflictos_1[['ID (for name)', 'Name and abbreviation', 'CAS no.', 'EC / List no.', 'Status', 'Status year', 'Appears on lists' ]]

Conflictos en Edlist_1:


,ID (for name),Name and abbreviation,CAS no.,EC / List no.,Status,Status year,Appears on lists
133,141,Dicyclohexyl phthalate (DCHP),84-61-7,201-545-9,Concluded ED in SEV,2023,"List I, List II"
134,141,Dicyclohexyl phthalate (DCHP),84-61-7,201-545-9,Health candidate list,2018,"List I, List II"
135,141,Dicyclohexyl phthalate (DCHP),84-61-7,201-545-9,Corap List,2017,"List I, List II"


Todos estos ingredientes han de ir en la Lista I porque ya se han demostrado que son ED.

Vemos que en realidad se trata solo de 1 ingrediente, pero hay varias líneas con información duplicada salvo la de Status y Status year, que para el objeto de estudio no es relevante.

Posiblemente pasará esto mismo con otros ingredientes, así que habrá que estudiar esto mismo más adelante.

In [12]:
# Vemos los casos concreto de edlist_2
conflictos_2 = edlist_2[edlist_2['Appears on lists'].str.contains(",", na=False)]
print("Conflictos en Edlist_2:")
conflictos_2[['ID (for name)', 'Name and abbreviation', 'CAS no.', 'EC / List no.', 'Status', 'Year', 'Appears on lists' ]]

Conflictos en Edlist_2:


,ID (for name),Name and abbreviation,CAS no.,EC / List no.,Status,Year,Appears on lists
1,141,Dicyclohexyl phthalate (DCHP),84-61-7,201-545-9,Corap List,2017,"List I, List II"
2,141,Dicyclohexyl phthalate (DCHP),84-61-7,201-545-9,Health candidate list,2018,"List I, List II"
3,141,Dicyclohexyl phthalate (DCHP),84-61-7,201-545-9,Concluded ED in SEV,2023,"List I, List II"
6,165,Propyl 4-hydroxybenzoate; Propylparaben,94-13-3,202-307-7,Corap List,2015,"List II, List III"
7,165,Propyl 4-hydroxybenzoate; Propylparaben,94-13-3,202-307-7,List III National Authority evaluation,"2022, 2023","List II, List III"
8,165,Propyl 4-hydroxybenzoate; Propylparaben,94-13-3,202-307-7,Concluded ED in SEV,2023,"List II, List III"
9,165,Propyl 4-hydroxybenzoate; Propylparaben,94-13-3,202-307-7,Commission EDC list,2019,"List II, List III"
10,165,Propyl 4-hydroxybenzoate; Propylparaben,94-13-3,202-307-7,Classification proposed,2024,"List II, List III"
11,166,Resorcinol,108-46-3,203-585-2,Corap List,2019,"List II, List III"
12,166,Resorcinol,108-46-3,203-585-2,Proposed for Candidate List,2020,"List II, List III"


De los 25 casos -> 3 deberían ser de la List I y el resto quedarse en la List II.

En realidad solo hay 6 ingredientes en conflicto, pero hay duplicidad.

In [13]:
# Vemos los casos concreto de edlist_3
conflictos_3 = edlist_3[edlist_3['Also appears on lists'].str.contains(",", na=False)]
print("Conflictos en Edlist_3:")
conflictos_3[['ID (for name)', 'Name and abbreviation', 'CAS no.', 'EC / List no.', 'Status', 'Year', 'Also appears on lists' ]]

Conflictos en Edlist_3:


,ID (for name),Name and abbreviation,CAS no.,EC / List no.,Status,Year,Also appears on lists
9,165,Propyl 4-hydroxybenzoate; Propylparaben,94-13-3,202-307-7,Corap List,2015,"List II, List III"
10,165,Propyl 4-hydroxybenzoate; Propylparaben,94-13-3,202-307-7,List III National Authority evaluation,"2022, 2023","List II, List III"
11,165,Propyl 4-hydroxybenzoate; Propylparaben,94-13-3,202-307-7,Concluded ED in SEV,2023,"List II, List III"
12,165,Propyl 4-hydroxybenzoate; Propylparaben,94-13-3,202-307-7,Commission EDC list,2019,"List II, List III"
13,165,Propyl 4-hydroxybenzoate; Propylparaben,94-13-3,202-307-7,Classification proposed,2024,"List II, List III"
14,166,Resorcinol,108-46-3,203-585-2,Corap List,2019,"List II, List III"
15,166,Resorcinol,108-46-3,203-585-2,Proposed for Candidate List,2020,"List II, List III"
16,166,Resorcinol,108-46-3,203-585-2,Classification proposed,2024,"List II, List III"
17,166,Resorcinol,108-46-3,203-585-2,Commission EDC list,2019,"List II, List III"
18,166,Resorcinol,108-46-3,203-585-2,List III National Authority evaluation,2025,"List II, List III"


De los 22 casos en conflicto -> todos deberían ser de la List II.

En realidad solo hay 5 ingredientes en conflicto, pero hay duplicidad.

**Renombramos categorías utilizando mapeo según el criterio establecido en el enunciado**

In [14]:
mapeo_categorias = {
    'List I, List II': 'List I',
    'List I, List II, List III': 'List I',
    'List II, List III': 'List II',
    'List I, List III': 'List I' # Esta combinación no existe actualmente pero se deja por si en algún futuro se da el caso.
}

edlist_1['Appears on lists'] = edlist_1['Appears on lists'].replace(mapeo_categorias)
edlist_2['Appears on lists'] = edlist_2['Appears on lists'].replace(mapeo_categorias)
edlist_3['Also appears on lists'] = edlist_3['Also appears on lists'].replace(mapeo_categorias)

In [15]:
# Comprobamos "Appears on lists"
print("Edlist_1:")
print(edlist_1['Appears on lists'].value_counts(dropna=False))
print("\nEdlist_2:")
print(edlist_2['Appears on lists'].value_counts(dropna=False))
print("\nEdlist_3:")
print(edlist_3['Also appears on lists'].value_counts(dropna=False))

Edlist_1:
Appears on lists
List I    226
Name: count, dtype: int64

Edlist_2:
Appears on lists
List II    101
List I       3
Name: count, dtype: int64

Edlist_3:
Also appears on lists
List II     22
List III    11
Name: count, dtype: int64


Se han clasificado como queriamos.

## 4. Unión de tablas

Vamos a unir las tablas y nos vamos a quedar con todas las filas, porque no sabemos qué utilidad puedan tener más adelante.

En este punto podríamos eliminar duplicados, pero como seguramente hayan tambien duplicados no solo de la misma tabla sino entre distintas tablas, para no hacer el trabajo dos veces, dejamos ese análisis para después.

Antes de realizar la unicón vamos a:
- Unificar los nombres de columnas en todas las tablas para no tener problemas.
- Añadir una columna que nos diga la fuente de origen de la tabla

In [16]:
# renombramos columnas
edlist_1 = edlist_1.rename(columns={'Status year': 'Year'})
edlist_3 = edlist_3.rename(columns = {'Also appears on lists': 'Appears on lists'})

In [17]:
# añadimos columna
edlist_1['Fuente_original'] = 'Edlist_1'
edlist_2['Fuente_original'] = 'Edlist_2'
edlist_3['Fuente_original'] = 'Edlist_3'

In [18]:
# Comprobamos una de ellas
edlist_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226 entries, 0 to 225
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ID (for name)          226 non-null    int64 
 1   Name and abbreviation  226 non-null    object
 2   CAS no.                211 non-null    object
 3   EC / List no.          166 non-null    object
 4   Health effects         64 non-null     object
 5   Environmental effects  182 non-null    object
 6   Status                 226 non-null    object
 7   Regulatory Field       222 non-null    object
 8   Year                   226 non-null    int64 
 9   Appears on lists       226 non-null    object
 10  Fuente_original        226 non-null    object
dtypes: int64(2), object(9)
memory usage: 19.6+ KB


In [19]:
# Ordenamos las 3 listas por ID (for name)
edlist_1 = edlist_1.sort_values( 'ID (for name)')
edlist_2 = edlist_2.sort_values( 'ID (for name)')
edlist_3 = edlist_3.sort_values( 'ID (for name)')

In [20]:
# Concatenamos
edlist_completa = pd.concat([edlist_1, edlist_2, edlist_3], ignore_index=True)

In [21]:
edlist_completa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363 entries, 0 to 362
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ID (for name)          363 non-null    int64 
 1   Name and abbreviation  363 non-null    object
 2   CAS no.                344 non-null    object
 3   EC / List no.          301 non-null    object
 4   Health effects         172 non-null    object
 5   Environmental effects  261 non-null    object
 6   Status                 363 non-null    object
 7   Regulatory Field       341 non-null    object
 8   Year                   363 non-null    object
 9   Appears on lists       363 non-null    object
 10  Fuente_original        363 non-null    object
dtypes: int64(1), object(10)
memory usage: 31.3+ KB


In [22]:
# Vemos que el índice se ha reseteado correctamente y hay continuidad del mismo de la List I a la List II
edlist_completa.iloc[225:227]

,ID (for name),Name and abbreviation,CAS no.,EC / List no.,Health effects,Environmental effects,Status,Regulatory Field,Year,Appears on lists,Fuente_original
225,787,"20-(nonylphenoxy)-3,6,9,12,15,18-hexaoxaicosan...",27177-03-3,248-292-0,NaN,Yes,Environmental authorisation list,REACH,2017,List I,Edlist_1
226,122,"6,6'-di-tert-butyl-4,4'-thiodi-m-cresol",96-69-5,202-525-2,Yes,NaN,Corap List,REACH,2015,List II,Edlist_2


In [23]:
# Entre la List II y la III ocurre lo mismo
edlist_completa.iloc[329:331]

,ID (for name),Name and abbreviation,CAS no.,EC / List no.,Health effects,Environmental effects,Status,Regulatory Field,Year,Appears on lists,Fuente_original
329,817,"Tris(2-ethylhexyl) benzene-1,2,4-tricarboxylate",3319-31-1,222-020-0,NaN,Yes,Corap List,REACH,2012,List II,Edlist_2
330,123,Tris(methylphenyl)phosphate,1330-78-5,215-548-8,Yes,NaN,List III National Authority evaluation,NaN,2020,List III,Edlist_3


## 5. Tratamiento de Duplicados

Recordemos que solo nos es necesario tener un ingrediente, aunque aparezca en dos listas.

In [24]:
# Comprobamos nulos
edlist_completa.isna().sum()

ID (for name)              0
Name and abbreviation      0
CAS no.                   19
EC / List no.             62
Health effects           191
Environmental effects    102
Status                     0
Regulatory Field          22
Year                       0
Appears on lists           0
Fuente_original            0
dtype: int64

In [25]:
# Para hacer la comparativa de duplicados vamos a elegir la variable "Name and abbreviation" pues es única para cada ingrediente y no tiene nulos.
duplicados = edlist_completa[edlist_completa.duplicated(subset='Name and abbreviation', keep=False)].sort_values(by=['ID (for name)']) 
duplicados.iloc[84:92]

,ID (for name),Name and abbreviation,CAS no.,EC / List no.,Health effects,Environmental effects,Status,Regulatory Field,Year,Appears on lists,Fuente_original
283,249,Reaction products of phosphoryl trichloride an...,1244733-77-4,807-935-0,Yes,Yes,Corap List,REACH,2022,List II,Edlist_2
355,249,Reaction products of phosphoryl trichloride an...,1244733-77-4,807-935-0,Yes,NaN,List III National Authority evaluation,NaN,2024,List II,Edlist_3
353,249,Reaction products of phosphoryl trichloride an...,1244733-77-4,807-935-0,Yes,Yes,Corap List,REACH,2022,List II,Edlist_3
354,249,Reaction products of phosphoryl trichloride an...,1244733-77-4,807-935-0,Yes,NaN,Concluded ED in SEV,REACH,2023,List II,Edlist_3
356,249,Reaction products of phosphoryl trichloride an...,1244733-77-4,807-935-0,Yes,NaN,Classification proposed,CLH,2024,List II,Edlist_3
286,249,Reaction products of phosphoryl trichloride an...,1244733-77-4,807-935-0,Yes,NaN,Classification proposed,CLH,2024,List II,Edlist_2
285,249,Reaction products of phosphoryl trichloride an...,1244733-77-4,807-935-0,Yes,NaN,List III National Authority evaluation,NaN,2024,List II,Edlist_2
284,249,Reaction products of phosphoryl trichloride an...,1244733-77-4,807-935-0,Yes,NaN,Concluded ED in SEV,REACH,2023,List II,Edlist_2


Hemos revisado la lista de duplicados, hay más de 250 registros duplicados. 

Mostramos el ejemplo para el ingrediente con ID(for name) : 249 

Este tiene varias duplicidades:
1. Duplicidad inter lista: vemos que en la Edlist_2 viene el elemento 4 veces, como hemos dicho eso pasa porque hay un hitórico de Status.
2. Duplicidad entre listas: vemos que en la Edlist_3 aparecía 4 veces y ponía que aparecía en la lista II a la que le dabamos prioridad, porque sigue en estudio por la UE.

En este caso, nos quedaremos con el Status year más reciente, pero que nos de más información. 

In [26]:
# Consultamos duplicados por las variables más relevantes
duplicados = edlist_completa[edlist_completa.duplicated(subset=['ID (for name)','Name and abbreviation', 'CAS no.', 'EC / List no.'], keep='first')].sort_values(by=['ID (for name)']) 
duplicados.count()

ID (for name)            151
Name and abbreviation    151
CAS no.                  143
EC / List no.            136
Health effects            75
Environmental effects    101
Status                   151
Regulatory Field         135
Year                     151
Appears on lists         151
Fuente_original          151
dtype: int64

151 sería el número de registros a eliminar

In [27]:
# Antes comprobmaso si el año es numérico
edlist_completa['Year'].info ()

<class 'pandas.core.series.Series'>
RangeIndex: 363 entries, 0 to 362
Series name: Year
Non-Null Count  Dtype 
--------------  ----- 
363 non-null    object
dtypes: object(1)
memory usage: 3.0+ KB


In [28]:
# Tambien comprobamos que hay un solo valor para cada año
edlist_completa['Year'].value_counts()

Year
2017          82
2024          46
2013          40
2019          34
2021          28
2023          23
2012          22
2022          18
2020          15
2025          13
2014          10
2018           9
2015           9
2016           7
2022, 2023     2
2027           2
2011           1
2026           1
2020, 2021     1
Name: count, dtype: int64

In [29]:
# Renombramos los años quendono con el año más reciente
año = {'2022, 2023': '2023',
        '2020, 2021': '2021'}

edlist_completa['Year'] = edlist_completa['Year'].replace(año)


In [30]:
# Convertimos la variable en numérica
edlist_completa['Year'] = pd.to_numeric(edlist_completa['Year'])

In [31]:
print(edlist_completa['Year'].value_counts()) 

Year
2017    82
2024    46
2013    40
2019    34
2021    29
2023    25
2012    22
2022    18
2020    15
2025    13
2014    10
2018     9
2015     9
2016     7
2027     2
2011     1
2026     1
Name: count, dtype: int64


Avanzando en el código nos dimos cuenta que si solo nos quedamos con el año más reciente, la variable Regulatory Field se quedan sin información.

Por ello vamos a añadir una variable boleanas para ver si hay nulos.

Con ello, al eliminar duplicados tendremos en cuenta estas variable que son relevantes y de las que necesitaremos información más adelante y nuestro dataframe no se quedará con nulos innecesariamente.

In [32]:
# Eliminamos los duplicados por las variables más relantes y ordenando por las otras que pierdan información
edlist_final = edlist_completa.drop_duplicates(
    subset=['ID (for name)','Name and abbreviation', 'CAS no.', 'EC / List no.'], keep='first').sort_values(by = ['Year', 'Regulatory Field', 'Environmental effects','Health effects'],
    ascending=[ True, False, False, False ])
# Ordenamos de nuevo por "ID (for name)"
edlist_final= edlist_final.sort_values(by='ID (for name)')


In [33]:
# Comprobamos que efectivamente, el caso 249 se ha quedado con el que más información da y el más reciente
edlist_final.iloc[54:57] 

,ID (for name),Name and abbreviation,CAS no.,EC / List no.,Health effects,Environmental effects,Status,Regulatory Field,Year,Appears on lists,Fuente_original
282,246,"Esterification products of 1,3-dioxo-2-benzofu...",NaN,941-303-6,Yes,Yes,Corap List,REACH,2027,List II,Edlist_2
283,249,Reaction products of phosphoryl trichloride an...,1244733-77-4,807-935-0,Yes,Yes,Corap List,REACH,2022,List II,Edlist_2
287,250,Tris[2-chloro-1-(chloromethyl)ethyl] phosphate,13674-87-8,237-159-2,Yes,NaN,Classification proposed,CLH,2024,List II,Edlist_2


**Eliminamos varibles creadas para la ordenación**

In [34]:
edlist_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 212 entries, 226 to 329
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ID (for name)          212 non-null    int64 
 1   Name and abbreviation  212 non-null    object
 2   CAS no.                201 non-null    object
 3   EC / List no.          165 non-null    object
 4   Health effects         97 non-null     object
 5   Environmental effects  160 non-null    object
 6   Status                 212 non-null    object
 7   Regulatory Field       206 non-null    object
 8   Year                   212 non-null    int64 
 9   Appears on lists       212 non-null    object
 10  Fuente_original        212 non-null    object
dtypes: int64(2), object(9)
memory usage: 19.9+ KB


## 6. Tratamiento de Nulos de determinadas variables.

La variables **Environmental effects** y **Health effects** tiene como valor NAN los ceros, por tanto vamos a darles valor : 0-1
No tratamos el resto de nulos, porque eso ya es parte del EDA y previo a esto hay que tratarlo con más dataframes.

In [35]:
# Mapear 'Yes' a 1, y el resto a 0 (incluye NaN)
edlist_final['Health effects'] = (edlist_final['Health effects'] == 'Yes').astype(int)
edlist_final['Environmental effects'] = (edlist_final['Environmental effects'] == 'Yes').astype(int)


In [36]:
# Como hemos mencionado antes esta es la combinación en la que el dataframe se queda con los menores nulos posibles.
edlist_final.isnull().sum()

ID (for name)             0
Name and abbreviation     0
CAS no.                  11
EC / List no.            47
Health effects            0
Environmental effects     0
Status                    0
Regulatory Field          6
Year                      0
Appears on lists          0
Fuente_original           0
dtype: int64

## 7. Variables "CAS no." y "EC / List no."

Estas son variables comunes con el dataframe ECHA y son las claves por la que vamos a realizar la unión de tablas con otra fuente de datos.

Se ha detectado que hay registros con varios "CAS no." y "EC / List no." vamos a investigar cuáles son para darle tratamiento antes de unir.

In [37]:
# Filtramos filas con más de un CAS no. 
edlist_final[edlist_final['CAS no.'].str.contains(r'[/;,]', regex= True, na=False)]

,ID (for name),Name and abbreviation,CAS no.,EC / List no.,Health effects,Environmental effects,Status,Regulatory Field,Year,Appears on lists,Fuente_original
249,179,Cyclomethicone,"69430-24-6, 556-67-2, 541-02-6, 540-97-6","614-966-1, 209-136-7, 208-764-9, 208-762-8",1,0,Commission EDC list,Cosmetics,2019,List II,Edlist_2
255,187,"1-(5,6,7,8-tetrahydro-3,5,5,6,8,8-hexamethyl-2...","21145-77-7, 1506-02-1","244-240-6, 216-133-4",0,1,Corap List,REACH,2020,List II,Edlist_2
294,264,2-tert-butyl-4-methoxyphenol (BHA),"25013-16-5, 121-00-6","246-563-8, 204-442-7",1,1,Corap List,REACH,2015,List II,Edlist_2
362,744,"Lithium salts (lithium chloride, lithium carbo...","7447-41-8, 554-13-2, 1310-65-2","231-212-3, 209-062-5, 215-183-4",1,0,List III National Authority evaluation,NaN,2024,List III,Edlist_3


Vemos que en CAS no. hay 4 registros que tienen un varios CAS no.

Ahora veremos en EC / List no. coinciden los 4 y se añade uno más.

In [38]:
# Filtramos filas con más de un EC / List no.
edlist_final[edlist_final['EC / List no.'].str.contains(r'[/;,]', regex= True, na=False)]

,ID (for name),Name and abbreviation,CAS no.,EC / List no.,Health effects,Environmental effects,Status,Regulatory Field,Year,Appears on lists,Fuente_original
249,179,Cyclomethicone,"69430-24-6, 556-67-2, 541-02-6, 540-97-6","614-966-1, 209-136-7, 208-764-9, 208-762-8",1,0,Commission EDC list,Cosmetics,2019,List II,Edlist_2
255,187,"1-(5,6,7,8-tetrahydro-3,5,5,6,8,8-hexamethyl-2...","21145-77-7, 1506-02-1","244-240-6, 216-133-4",0,1,Corap List,REACH,2020,List II,Edlist_2
290,254,Oligomerisation and alkylation reaction produc...,68512-30-1,"270-966-8, 700-960-7",0,1,Corap List,REACH,2012,List II,Edlist_2
294,264,2-tert-butyl-4-methoxyphenol (BHA),"25013-16-5, 121-00-6","246-563-8, 204-442-7",1,1,Corap List,REACH,2015,List II,Edlist_2
362,744,"Lithium salts (lithium chloride, lithium carbo...","7447-41-8, 554-13-2, 1310-65-2","231-212-3, 209-062-5, 215-183-4",1,0,List III National Authority evaluation,NaN,2024,List III,Edlist_3


In [39]:
# Separamos CAS múltiples por coma en una lista
edlist_final['CAS_list'] = edlist_final['CAS no.'].str.split(r'[,;/]')
edlist_final['EC_list'] = edlist_final['EC / List no.'].str.split(r'[,;/]')
edlist_final[['ID (for name)','CAS_list','EC_list' ]].iloc[27:33] # localizamos algunos resultados

,ID (for name),CAS_list,EC_list
249,179,"[69430-24-6, 556-67-2, 541-02-6, 540-97-6]","[614-966-1, 209-136-7, 208-764-9, 208-762-8]"
29,181,[115-86-6],[204-112-2]
250,182,[52918-63-5],[258-256-6]
251,185,[53988-10-6],[258-904-8]
253,186,[61617-00-3],[262-872-0]
255,187,"[21145-77-7, 1506-02-1]","[244-240-6, 216-133-4]"


Descomponemos cada elemento de la lista de CAS_list y de EC_list. Al hacerlo se van a crear combinaciones entre ambas porque se va a descomponer hasta que cada una tenga solo un elemento.

De est manera si CAS_list tiene 4 elementos se tendrían que crear 4 filas para cada CAST_list pero si además EC_list tiene otros 4 elementos, no serían 8, sino 4^4 =16 porque se combinan los índices de las listas, como es en el caso del ID(for name ): 179

In [40]:
# Descomponemos fila por cada CAS individual
edlist_final = edlist_final.explode('CAS_list')
# Descomponemos fila por cada EC individual
edlist_final = edlist_final.explode('EC_list')

Comprobamos el resultado y vemos que se han realizado correctamente todas las combinaciones

In [41]:
# Vemos resultados de 'EC / List no.' porque incluye tambien los resultados de 'CAS no.'
consulta = edlist_final[edlist_final['EC / List no.'].str.contains(r'[/;,]', regex= True,na=False)]
consulta[['Name and abbreviation', 'CAS no.', 'EC / List no.', 'CAS_list', 'EC_list']]. head(16)

,Name and abbreviation,CAS no.,EC / List no.,CAS_list,EC_list
249,Cyclomethicone,"69430-24-6, 556-67-2, 541-02-6, 540-97-6","614-966-1, 209-136-7, 208-764-9, 208-762-8",69430-24-6,614-966-1
249,Cyclomethicone,"69430-24-6, 556-67-2, 541-02-6, 540-97-6","614-966-1, 209-136-7, 208-764-9, 208-762-8",69430-24-6,209-136-7
249,Cyclomethicone,"69430-24-6, 556-67-2, 541-02-6, 540-97-6","614-966-1, 209-136-7, 208-764-9, 208-762-8",69430-24-6,208-764-9
249,Cyclomethicone,"69430-24-6, 556-67-2, 541-02-6, 540-97-6","614-966-1, 209-136-7, 208-764-9, 208-762-8",69430-24-6,208-762-8
249,Cyclomethicone,"69430-24-6, 556-67-2, 541-02-6, 540-97-6","614-966-1, 209-136-7, 208-764-9, 208-762-8",556-67-2,614-966-1
249,Cyclomethicone,"69430-24-6, 556-67-2, 541-02-6, 540-97-6","614-966-1, 209-136-7, 208-764-9, 208-762-8",556-67-2,209-136-7
249,Cyclomethicone,"69430-24-6, 556-67-2, 541-02-6, 540-97-6","614-966-1, 209-136-7, 208-764-9, 208-762-8",556-67-2,208-764-9
249,Cyclomethicone,"69430-24-6, 556-67-2, 541-02-6, 540-97-6","614-966-1, 209-136-7, 208-764-9, 208-762-8",556-67-2,208-762-8
249,Cyclomethicone,"69430-24-6, 556-67-2, 541-02-6, 540-97-6","614-966-1, 209-136-7, 208-764-9, 208-762-8",541-02-6,614-966-1
249,Cyclomethicone,"69430-24-6, 556-67-2, 541-02-6, 540-97-6","614-966-1, 209-136-7, 208-764-9, 208-762-8",541-02-6,209-136-7


Dejamos las variables nuevas así aunque sean el mismo ingrediente duplicado, porque luego queremos unir las tablas y entre más información tenga la variable mejor será la unión entre tablas.

In [42]:
# Eliminamos 'CAS no.' y 'EC / List no.' porque ya no nos sirven.
edlist_final.drop(columns='CAS no.', inplace=True)
edlist_final.drop(columns='EC / List no.', inplace=True)

In [43]:
#renombrar variables con el nombre antiguo
edlist_final = edlist_final.rename(columns={'CAS_list': 'CAS Number', 'EC_list':'EC Number'})

Comprobamos que no hay valores extraños en las variables "EC no." y "CAS no." antes de exportar

In [44]:
edlist_final[edlist_final['CAS Number'] == '-']

,ID (for name),Name and abbreviation,Health effects,Environmental effects,Status,Regulatory Field,Year,Appears on lists,Fuente_original,CAS Number,EC Number
293,256,"reaction mass of 1-(1,2,3,4,5,6,7,8-octahydro-...",1,1,Corap List,REACH,2017,List II,Edlist_2,-,915-730-3


In [45]:
edlist_final['CAS Number'] = edlist_final['CAS Number'].replace(['-', '', ' '], np.nan)
edlist_final['EC Number'] = edlist_final['EC Number'].replace(['-', '', ' '], np.nan)

Para no crear conflictos en la unión, tenemos que comprobar que las 2 varibles clave: "CAS no." y "EC no." no tengan nulos.

In [46]:
# Comprobamos la condición de que "CAS no." y "EC no." sean nulos.

edlist_final[(edlist_final['CAS Number'].isna() & edlist_final['EC Number'].isna())]

,ID (for name),Name and abbreviation,Health effects,Environmental effects,Status,Regulatory Field,Year,Appears on lists,Fuente_original,CAS Number,EC Number
315,393,Mono- and di-phthalate esters with linear and/...,1,1,Classification proposed,CLH,2024,List II,Edlist_2,NaN,NaN
210,766,"20-(isononylphenoxy)-3,6,9,12,15,18-hexaoxaico...",0,1,Environmental candidate list,REACH,2013,List I,Edlist_1,NaN,NaN
328,813,Benzotriazole; sodium 1H-benzotriazolide; pota...,0,1,Classification proposed,CLH,2025,List II,Edlist_2,NaN,NaN


In [47]:
# Filtramos los registros que cumplen la condición que "CAS no." y "EC no." sean NO nulos.

edlist_final = edlist_final[~(edlist_final['CAS Number'].isna() & edlist_final['EC Number'].isna())]

In [48]:
edlist_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 239 entries, 226 to 329
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ID (for name)          239 non-null    int64 
 1   Name and abbreviation  239 non-null    object
 2   Health effects         239 non-null    int64 
 3   Environmental effects  239 non-null    int64 
 4   Status                 239 non-null    object
 5   Regulatory Field       225 non-null    object
 6   Year                   239 non-null    int64 
 7   Appears on lists       239 non-null    object
 8   Fuente_original        239 non-null    object
 9   CAS Number             230 non-null    object
 10  EC Number              195 non-null    object
dtypes: int64(4), object(7)
memory usage: 22.4+ KB


## 8. Exportamos base de datos limpia
Posteriormente vamos a unir esta tabla con la tabla ECHA y CosIng.

In [49]:
# Reiniciamos ínfice
edlist_final.reset_index(drop=True, inplace=True)
edlist_clean= edlist_final

In [50]:
# CSV
# edlist_clean.to_csv(""../../data/processed/notebooks/edlist_clean.csv", index=False)

# Parquet
edlist_clean.to_parquet("../../data/processed/notebooks/edlist_clean.parquet", index=False)